In [2]:
import pandas as pd

In [3]:

df_historical_data=pd.read_csv('uefa_euros_historical_data.csv')
df_fixture=pd.read_csv('uefa_euros_fixture.csv')


<h2>1 Data Cleaning</h2>

<h3>1.1 strip spaces</h3>

In [4]:
df_fixture['home']=df_fixture['home'].str.strip()
df_fixture['away']=df_fixture['away'].str.strip()

<h3>check score validity</h3>
<h4>a.e.t => after extra time 
 "a.e.t./g.g." typically stands for "after extra time/goals scored in extra time," while "a.e.t./s.g." stands for "after extra time/penalty shootout.
</h4>

In [5]:
import re

pattern_only_score = r'^\d+[\u2013\u2212−]\d+$'
pattern_score_extra_time_goal=r'^\d+[\u2013\u2212−]\d+(?: \([a-z./ ]+\))?$'

# Apply the regular expression to each element in the 'score' column
matches = df_historical_data['score'].apply(lambda x: bool(re.match(pattern_only_score, x)))

# Filter the DataFrame based on the matches
unmatched_rows = df_historical_data[~matches]

# Print the rows where the score format matches the pattern
print(unmatched_rows)


                home                               score             away  \
3      Soviet Union                         2–1 (a.e.t.)       Yugoslavia   
4             Spain                         2–1 (a.e.t.)          Hungary   
6           Hungary                         3–1 (a.e.t.)          Denmark   
8             Italy   0–0 (a.e.t.)Italy won on coin toss     Soviet Union   
11            Italy                         1–1 (a.e.t.)       Yugoslavia   
17   Czechoslovakia                         3–1 (a.e.t.)      Netherlands   
18       Yugoslavia                         2–4 (a.e.t.)     West Germany   
19      Netherlands                         3–2 (a.e.t.)       Yugoslavia   
20   Czechoslovakia                         2–2 (a.e.t.)     West Germany   
47           France                         3–2 (a.e.t.)         Portugal   
48          Denmark                         1–1 (a.e.t.)            Spain   
78      Netherlands                         2–2 (a.e.t.)          Denmark   

In [6]:
df_historical_data.drop(8) # won on coin toss

,home,score,away,year
0,France,4–5,Yugoslavia,1960
1,Czechoslovakia,0–3,Soviet Union,1960
2,Czechoslovakia,2–0,France,1960
3,Soviet Union,2–1 (a.e.t.),Yugoslavia,1960
4,Spain,2–1 (a.e.t.),Hungary,1964
...,...,...,...,...
332,Czech Republic,1–2,Denmark,2020
333,Ukraine,0–4,England,2020
334,Italy,1–1 (a.e.t.),Spain,2020
335,England,2–1 (a.e.t.),Denmark,2020


<h3>Have only score line</h3>

In [7]:
df_historical_data['score']

0               4–5
1               0–3
2               2–0
3      2–1 (a.e.t.)
4      2–1 (a.e.t.)
           ...     
332             1–2
333             0–4
334    1–1 (a.e.t.)
335    2–1 (a.e.t.)
336    1–1 (a.e.t.)
Name: score, Length: 337, dtype: object

In [8]:
df_historical_data['score'] = df_historical_data['score'].str.replace(r'[^\d\u2013\u2212−-]', '', regex=True)


In [9]:
df_historical_data['home']=df_historical_data['home'].str.strip()
df_historical_data['away']=df_historical_data['away'].str.strip()

<h3>spliting score</h3>

In [10]:

df_historical_data['home_score'] = df_historical_data['score'].apply(lambda x: x.split('–')[0])
df_historical_data['away_score'] = df_historical_data['score'].apply(lambda x: x.split('–')[1])

#df_historical_data[['home_goal','away_goal']]=df_historical_data['score'].str.split('-',expand=True)

<h3>deleting score column</h3>

In [11]:
df_historical_data.drop('score',axis=1, inplace=True)


<h2>renaming column and change data type</h3>

In [12]:
df_historical_data.rename(columns={'home':'HomeTeam','away':'AwayTeam','year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'home_score': int, 'away_score': int, 'Year': int})



In [13]:
df_historical_data.dtypes


HomeTeam      object
AwayTeam      object
Year           int32
home_score     int32
away_score     int32
dtype: object

In [14]:
#creating new column totalgoals
df_historical_data['TotalGoals']=df_historical_data['home_score']+df_historical_data['away_score']

<h1>2 Exporting clean dataFrames</h1>

In [15]:
df_historical_data.to_csv('clean_uefa_euros_historical_data.csv',index=False)
df_fixture.to_csv('clean_uefa_euros_fixture.csv',index=False)


<h1>3 Extra Verification</h3>

In [16]:
years=[]
for i in range(1960,2021,4):
    years.append(i)

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))
#verified from  https://en.wikipedia.org/wiki/UEFA_European_Championship#Expansion_to_8_teams

1960 4
1964 4
1968 5
1972 4
1976 4
1980 14
1984 15
1988 15
1992 15
1996 31
2000 31
2004 31
2008 31
2012 31
2016 51
2020 51


In [17]:
df_historical_data

,HomeTeam,AwayTeam,Year,home_score,away_score,TotalGoals
0,France,Yugoslavia,1960,4,5,9
1,Czechoslovakia,Soviet Union,1960,0,3,3
2,Czechoslovakia,France,1960,2,0,2
3,Soviet Union,Yugoslavia,1960,2,1,3
4,Spain,Hungary,1964,2,1,3
...,...,...,...,...,...,...
332,Czech Republic,Denmark,2020,1,2,3
333,Ukraine,England,2020,0,4,4
334,Italy,Spain,2020,1,1,2
335,England,Denmark,2020,2,1,3
